<a href="https://colab.research.google.com/github/Sameeksha2597/AICourseProject/blob/main/03_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install lime shap datasets transformers torch --quiet
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from lime.lime_text import LimeTextExplainer
import shap
import numpy as np
import torch
from IPython.display import display, HTML
dataset = load_dataset("glue", "sst2", split="validation[:10]")

print("Loaded 10 samples from GLUE SST-2 validation set:\n")
for i, example in enumerate(dataset):
    print(f"{i+1}. {example['sentence']} (Label: {example['label']})")
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

classifier = pipeline("text-classification", model=model_name, return_all_scores=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["NEGATIVE", "POSITIVE"]
def predict_proba(texts):
    outputs = classifier(texts)
    probs = []
    for out in outputs:
        probs.append([out[0]["score"], out[1]["score"]])
    return np.array(probs)

print("Generating LIME explanations...")
print("\n")

explainer = LimeTextExplainer(class_names=labels)

for i in range(10):
    text = dataset[i]["sentence"]
    print(f"\nLIME → Sentence {i+1}: {text}")

    exp = explainer.explain_instance(text, predict_proba, num_features=8)

    html_file = f"lime_result_{i+1}.html"
    exp.save_to_file(html_file)

    display(HTML(exp.as_html()))
print("Generating SHAP explanations")

def f(x):
    inputs = tokenizer(list(x), return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        return torch.nn.functional.softmax(outputs.logits, dim=1).numpy()

explainer_shap = shap.Explainer(f, tokenizer)

for i in range(10):
    text = dataset[i]["sentence"]
    print(f"\nSHAP → Sentence {i+1}: {text}")
    shap_values = explainer_shap([text])
    shap.plots.text(shap_values[0])




Mounted at /content/drive
Loaded 10 samples from GLUE SST-2 validation set:

1. it 's a charming and often affecting journey .  (Label: 1)
2. unflinchingly bleak and desperate  (Label: 0)
3. allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker .  (Label: 1)
4. the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales .  (Label: 1)
5. it 's slow -- very , very slow .  (Label: 0)
6. although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women .  (Label: 1)
7. a sometimes tedious film .  (Label: 0)
8. or doing last year 's taxes with your ex-wife .  (Label: 0)
9. you do n't have to know about music to appreciate the film 's easygoing blend of comedy and romance .  (Label: 1)
10. in exactly 89 minutes , most of which passed as slowly as if i 'd been sitting naked on an igloo , formula 51 sank from quirky to jerky to utter turkey .  (L

Device set to use cpu


Generating LIME explanations...



LIME → Sentence 1: it 's a charming and often affecting journey . 
